<a href="https://colab.research.google.com/github/jkrue242/PortfolioOptimization/blob/main/portfolio_optimiztion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Portfolio Optimization using Deep Learning

#### Implementation and modification of [this paper](https://arxiv.org/pdf/2005.13665).

#### Modifications will include:
1. Use of different recursive networks (Transformer, GRU, LSTM)
2. Hyperparameter optimization
3. Additional input features
4. Loss function comparison between Sharpe and Sortino

Imports

In [2]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import warnings
from data_utils import PortfolioDataset
from transformer import PortfolioTransformer
from sharpe import SharpeLoss
warnings.filterwarnings('ignore')

ImportError: cannot import name 'PortfolioDataset' from 'data_utils' (/Users/josephkrueger/college/2025_spring/deep_learning/PortfolioOptimization/data_utils.py)

In our analysis, we will build a simple portfolio of 4 indices:
1. AGG (Agg. Bond ETF)
2. DBC (Commodity Index)
3. VTI (Vanguard Total Stock Index)
4. VIX (CBOE Volatility Index)

In [ ]:
# constants for dataset and training stuff
ETFS = ["VOO","QQQ","GLD"]
N_TICKERS = len(ETFS)

SAMPLE_DAYS = 50
ROLLING_AVGS = [10, 30, 50]
BATCH_SIZE = 64
START_DATE = '2010-01-01'
END_DATE ='2025-01-01'

# try to use cuda for training
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running notebook on {DEVICE}")

Pull data from yfinance. Features used:
1. Close Price
2. Returns (% change of close)
3. 10-day, 30-day, 50-day Rolling Averages

In [ ]:
ticker_df = yf.download(ETFS, start=START_DATE, end=END_DATE, group_by='ticker', auto_adjust=True)
features = []
for e in ETFS:
  data = ticker_df[e].copy()

  # returns features
  data['return'] = data['Close'].pct_change().fillna(0)
  data['cumulative_return'] = (1 + data['return']).cumprod() - 1

  # rolling average feature
  for i in ROLLING_AVGS:
    data[f"ma_{i}"] = data['Close'].rolling(i).mean().fillna(method='bfill')

  # build up feature list
  features.append(data[['return'] + ['cumulative_return'] + [f"ma_{i}" for i in ROLLING_AVGS]])

data = pd.concat(features, axis=1, keys=ETFS).dropna()
N_FEATURES = len(data.columns)//N_TICKERS

In [ ]:
print(f"Number of features: {N_FEATURES}")

In [ ]:
data.tail()

Plot all the features

In [ ]:
feature_names = data.columns.get_level_values(1).unique()

for feature_name in feature_names:
  # feature cross sections
  feature_data = data.xs(feature_name, level=1, axis=1)

  plt.figure(figsize=(14, 7))
  ax = plt.gca()

  feature_data.plot(ax=ax)

  plt.title(f'{feature_name} Over Time for All ETFs')
  plt.xlabel('Date')
  plt.ylabel(feature_name.replace('_', ' ').title())

  plt.legend(title='ETFs')

  plt.grid(True)
  plt.show()

Training

In [ ]:
# load data
dataset = PortfolioDataset(data, SAMPLE_DAYS)
data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# build model
model = PortfolioTransformer(
  seq_len=SAMPLE_DAYS,
  num_features=len(ETFS)*N_FEATURES,
  num_assets=len(ETFS),
  d_model=64,
  nhead=4,
  num_layers=2,
  dim_ff=128,
  dropout=0.1
).to(DEVICE)

# use neg sharpe as loss
opt = torch.optim.Adam(model.parameters(), lr=1e-4)
loss_fn = SharpeLoss()

for epoch in range(1, 200):
  model.train()
  total = 0

  # batch train
  for batch_idx, batch in enumerate(data_loader):
    X = batch['features'].to(DEVICE)
    R = batch['returns'].to(DEVICE)

    # zero out
    opt.zero_grad()

    # forward pass to predict weights
    w = model(X)

    # printing loss
    if batch_idx == 0:
      print(f"  Epoch {epoch:02d}, Batch {batch_idx:03d} - Predicted Weights (first sample):")
      weights_to_print = w[0].detach().cpu().numpy()
      for i, etf in enumerate(ETFS):
        print(f"    {etf}: {weights_to_print[i]:.4f}", end=" ")
      print()

    # backprop
    loss = loss_fn(w, R)
    loss.backward()
    opt.step()
    total += loss.item()

  l = total / len(data_loader)
  print(f"Epoch {epoch:02d}, Loss: {l:.4f}")

Plots

In [ ]:
# eval mode
model.eval()
predicted_weights = []
actual_returns = []
portfolio_returns = []
dates = []
all_dates = data.index[(SAMPLE_DAYS + 1):]

# zero grad for prediciton
with torch.no_grad():
  for i in range(len(dataset)):

    # single sample
    batch = dataset[i]
    X = batch['features'].unsqueeze(0).to(DEVICE)
    R_actual = batch['returns'].cpu().numpy()

    # portfolio weights
    weights = model(X).squeeze(0).cpu().numpy()

    # store weights, returns, dates
    predicted_weights.append(weights)
    actual_returns.append(R_actual)
    dates.append(all_dates[i])

    # portfolio return: dot product of weights and actual returns
    Rp_t = np.sum(weights * R_actual)
    portfolio_returns.append(Rp_t)

# to np
predicted_weights = np.array(predicted_weights)
actual_returns = np.array(actual_returns)
portfolio_returns = np.array(portfolio_returns)

# compound return
compounded_returns = (1 + portfolio_returns).cumprod()

cumulative_sharpe_ratios = []
for j in range(len(portfolio_returns)):
  returns_subset = portfolio_returns[:j+1]

  if len(returns_subset) > 1:
    mean_return = np.mean(returns_subset)
    std_return = np.std(returns_subset, ddof=0)

    if std_return != 0:
      # sharpe ratio no risk free
      sharpe = (mean_return / std_return)
    else:
      sharpe = 0
    cumulative_sharpe_ratios.append(sharpe)
  else:
    cumulative_sharpe_ratios.append(0)

cumulative_sharpe_ratios = np.array(cumulative_sharpe_ratios)

# feature cross section
cumulative_return_feature_data_for_plot = data.loc[dates].xs('cumulative_return', level=1, axis=1)

fig, axes = plt.subplots(3, 1, figsize=(14, 15), sharex=True)

# weights vs time
axes[0].set_title('Portfolio Weights Over Time (Model Predictions)')
axes[0].set_ylabel('Weight')
for i, etf in enumerate(ETFS):
  axes[0].plot(dates, predicted_weights[:, i], label=etf)
axes[0].legend(title='ETFs')
axes[0].grid(True)

# return vs time
axes[1].set_title('Cumulative Return Feature Over Time for Each ETF')
axes[1].set_ylabel('Cumulative Return')
cumulative_return_feature_data_for_plot.plot(ax=axes[1])
axes[1].legend(title='ETFs')
axes[1].grid(True)

# shapre vs time
axes[2].set_title('Cumulative Sharpe Ratio of Model Portfolio Over Time (Annualized)')
axes[2].set_ylabel('Sharpe Ratio')
axes[2].set_xlabel('Date')
axes[2].plot(dates, cumulative_sharpe_ratios)
axes[2].grid(True)


plt.tight_layout()
plt.show()